# Python Done Wrong

In [1]:
#https://github.com/ohadravid/poly-match/blob/main/poly_match_v1.py
from functools import cached_property
from typing import List, Tuple
import numpy as np
from dataclasses import dataclass

Point = np.array


@dataclass
class Polygon:
    x: np.array
    y: np.array
    _area: float = None

    @cached_property
    def center(self) -> np.array:
        centroid = np.array([self.x, self.y]).mean(axis=1)
        return centroid

    def area(self) -> float:
        if self._area is None:
            self._area = 0.5 * np.abs(
                np.dot(self.x, np.roll(self.y, 1)) - np.dot(self.y, np.roll(self.x, 1))
            )
        return self._area


def generate_one_polygon() -> Polygon:
    x = np.arange(0.0, 1.0, 0.1)
    y = np.sqrt(1.0 - x**2)
    return Polygon(x=x, y=y)


def generate_example() -> Tuple[List[Polygon], List[np.array]]:
    rng = np.random.RandomState(6)
    xs = np.arange(0.0, 100.0, 1.0)
    rng.shuffle(xs)

    ys = np.arange(0.0, 100.0, 1.0)
    rng.shuffle(ys)

    points = [np.array([x, y]) for x, y in zip(xs, ys)]

    ex_poly = generate_one_polygon()
    polygons = [
        Polygon(
            x=ex_poly.x + rng.randint(0.0, 100.0),
            y=ex_poly.y + rng.randint(0.0, 100.0),
        )
        for _ in range(1000)
    ]

    return polygons, points


def find_close_polygons(
    polygon_subset: List[Polygon], point: np.array, max_dist: float
) -> List[Polygon]:
    close_polygons = []
    for poly in polygon_subset:
        if np.linalg.norm(poly.center - point) < max_dist:
            close_polygons.append(poly)

    return close_polygons


def select_best_polygon(
    polygon_sets: List[Tuple[Point, List[Polygon]]]
) -> List[Tuple[Point, Polygon]]:
    best_polygons = []
    for point, polygons in polygon_sets:
        best_polygon = polygons[0]

        for poly in polygons:
            if poly.area() < best_polygon.area():
                best_polygon = poly

        best_polygons.append((point, best_polygon))

    return best_polygons


def main(polygons: List[Polygon], points: np.ndarray) -> List[Tuple[Point, Polygon]]:
    max_dist = 10.0
    polygon_sets = []
    for point in points:
        close_polygons = find_close_polygons(polygons, point, max_dist)

        if len(close_polygons) == 0:
            continue

        polygon_sets.append((point, close_polygons))

    best_polygons = select_best_polygon(polygon_sets)

    return best_polygons



polygons, points = generate_example()


In [2]:
%%timeit
best_polygons = main(polygons, points)

340 ms ± 529 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Numpy Done Wrong

In [3]:
# https://github.com/ohadravid/poly-match/blob/main/poly_match_v1_5_vectorized.py
from functools import cache, cached_property
from typing import Dict, List, Tuple
import numpy as np
from dataclasses import dataclass


Point = np.array


@dataclass
class Polygon:
    x: np.array
    y: np.array
    _area: float = None

    @cached_property
    def center(self) -> np.array:
        centroid = np.array([self.x, self.y]).mean(axis=1)
        return centroid

    def area(self) -> float:
        if self._area is None:
            self._area = 0.5 * np.abs(
                np.dot(self.x, np.roll(self.y, 1)) - np.dot(self.y, np.roll(self.x, 1))
            )
        return self._area


def generate_one_polygon() -> Polygon:
    x = np.arange(0.0, 1.0, 0.1)
    y = np.sqrt(1.0 - x**2)
    return Polygon(x=x, y=y)


def generate_example() -> Tuple[List[Polygon], List[np.array]]:
    rng = np.random.RandomState(6)
    xs = np.arange(0.0, 100.0, 1.0)
    rng.shuffle(xs)

    ys = np.arange(0.0, 100.0, 1.0)
    rng.shuffle(ys)

    points = [np.array([x, y]) for x, y in zip(xs, ys)]

    ex_poly = generate_one_polygon()
    polygons = [
        Polygon(
            x=ex_poly.x + rng.randint(0.0, 100.0),
            y=ex_poly.y + rng.randint(0.0, 100.0),
        )
        for _ in range(1000)
    ]

    return polygons, points


def find_close_polygons(
    point_idx: int, polygon_to_points_dist: Dict[int, np.array], max_dist: float
) -> List[Polygon]:
    close_polygons = []
    for poly, points_dist in polygon_to_points_dist.items():
        if points_dist[point_idx] < max_dist:
            close_polygons.append(poly)

    return close_polygons


def select_best_polygon(
    polygon_sets: List[Tuple[Point, List[Polygon]]]
) -> List[Tuple[Point, Polygon]]:
    best_polygons = []
    for point, polygons in polygon_sets:
        best_polygon = polygons[0]

        for poly in polygons:
            if poly.area() < best_polygon.area():
                best_polygon = poly

        best_polygons.append((point, best_polygon))

    return best_polygons


def find_dist_to_points(polygon: Polygon, points: np.ndarray) -> np.ndarray:
    return np.linalg.norm(polygon.center - points, axis=1)


def main(polygons: List[Polygon], points: np.ndarray) -> List[Tuple[Point, Polygon]]:
    max_dist = 10.0
    polygon_to_points_dist = {}

    for polygon_idx, polygon in enumerate(polygons):
        polygon_to_points_dist[polygon_idx] = find_dist_to_points(polygon, points)
    #============
    polygon_sets = []
    #============
    for point_idx, point in enumerate(points):
        close_polygons_indices = find_close_polygons(point_idx, polygon_to_points_dist, max_dist)
        if len(close_polygons_indices) == 0:
            continue
        close_polygons = [polygons[idx] for idx in close_polygons_indices]
        polygon_sets.append((point, close_polygons))

    best_polygons = select_best_polygon(polygon_sets)

    return best_polygons

polygons, points = generate_example()


In [4]:
%%timeit
best_polygons = main(polygons, points)

46.5 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Numpy Done Right

In [5]:
from operator import itemgetter as itg
from collections import defaultdict as ddict
import numpy as np
from line_profiler import LineProfiler

#profile = LineProfiler()


def make_points_2D():
    X = np.arange(0, 100, dtype='float32')
    Y = X.copy()
    np.random.shuffle(X)
    np.random.shuffle(Y)
    return np.vstack([X, Y]).T


def make_ref_polygon():
    X_ref = np.arange(0, 1, 0.1, dtype="float32")
    Y_ref = 1 - X_ref**2
    return np.vstack([X_ref, Y_ref]).T


def make_polygons(ref_polygon=make_ref_polygon()):
    random_samples_2D = np.random.randint(0, 100, size=(1000, 2)).astype("float32")
    random_samples_2D[:10]
    # XY_ref.shape = (10, 2), and we want 1000 polygons
    # Transpose to allows for broadcasting
    shiftings = np.broadcast_to(ref_polygon, (1000, 10, 2)).transpose(1, 0, 2)
    polygons = (random_samples_2D + shiftings).transpose(1, 0, 2)
    return polygons


#@profile
def done_right(points, polygons):
    #=== Get point of center of each polygon
    # Take X.mean and Y.mean
    polygons_centeroid = polygons.mean(axis=-2)  # shape=(1000,)

    #=== Find distances bewteen polygons and points
    points_grid = np.broadcast_to(points, (1000, 100, 2)).transpose(1, 0, 2)  # shape=(100, 1000, 2)
    polygons_points_distances = np.linalg.norm( (polygons_centeroid - points_grid)**2 , axis=-1)  # shape=(100, 1000)
    # (X, Y) axis is merged. left only points and polygon axis
    polygons_points_distances = polygons_points_distances.transpose(1, 0)  # shape=(1000, 100)

    #=== But first need to calculate Areas for polygons
    polygons_x = polygons[:, :, 0]  # shape=(1000, 10)
    polygons_y = polygons[:, :, 1]  # shape=(1000, 10)
    polygons_areas = 0.5 * (
        (polygons_x * np.roll(polygons_y, 1, axis=-1)).sum(axis=-1)
        - (polygons_y * np.roll(polygons_x, 1, axis=-1)).sum(axis=-1)
    )    # shape=(1000,)
    #=== Find all polygons that is near by a certain point
    neighbour_points = np.where(
        polygons_points_distances < 10,  # shape=(1000, 100)
        np.broadcast_to(polygons_areas, (100, 1000)).T,  # shape=(1000, 100)
        0)  # shape=(1000, 100)
    best_pairs = np.argmax(neighbour_points, axis=0)  # shape=(100,)
    return best_pairs


np.random.seed(6)
mypoints = make_points_2D()
mypolygons = make_polygons()
#profile.print_stats(output_unit=0.001)

In [6]:
%%timeit
best_pairs = done_right(mypoints, mypolygons)

973 µs ± 9.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
print(f"Python Done Wrong vs Numpy Done Right : {340 /0.973}X")
print(f"Numpy  Done Wrong vs Numpy Done Right : {46.5 /0.973}X")

Python Done Wrong vs Numpy Done Right : 349.43473792394656X
Numpy  Done Wrong vs Numpy Done Right : 47.79033915724563X
